<div align="right"><em>Машинне навчання. Лабораторний практикум</em></div>


# Лабораторна робота №5
## Лінійна регресія, Lasso і RF-регресія*
*на прикладі задачі визначення якості вина
    
    
---

## Завдання

Необхідно заповнити пропущений код та дати відповідь на всі запитання сформульовані в лабораторній роботі. 

## Виконання завдання

In [3]:
# відключаємо попередження Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Будемо працювати з набором даних по якості білого вина (репозиторій UCI).**
**Завантажуємо дані.**

In [4]:
data = pd.read_csv('../data/winequality-white.csv', sep=';')

In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Відокремте цільову ознаку, розділіть навчальну вибірку у відношенні 7:3 (30% - під відкладену вибірку, нехай random_state=17) і відмасштабуйте дані за допомогою StandardScaler.**

In [18]:
y = data['quality']# Ваш код тут
X = data.drop('quality', axis='columns')

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17) # Ваш код тут
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Ваш код тут
X_holdout_scaled = scaler.transform(X_holdout) # Ваш код тут


## Лінійна регресія

**Навчіть просту лінійну регресію.**

In [19]:
linreg = LinearRegression()# Ваш код тут
linreg.fit(X_train, y_train) # Ваш код тут

LinearRegression()

**Запитання 1. (2 бали). Які середньоквадратичні помилки лінійної регресії на навчальній і відкладеній вибірках?**

In [20]:
y_pred_test = linreg.predict(X_holdout)
y_pred_train = linreg.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_holdout, y_pred_test)

print(f"Mean squared error (train): {mse_train:.3f}")  # Ваш код тут
print(f"Mean squared error (test): {mse_test:.3f}")  # Ваш код тут

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Подивіться на коефіцієнти моделі і відранжуйте ознаки за впливом на якість вина (врахуйте, що великі за модулем від'ємні значення коефіцієнтів теж говорять про сильний вплив). Створіть для цього новий невеликий DataFrame.**


**Запитання 2. (1 бал). Яку ознаку лінійна регресія вважає найбільш впливаючою на якість вина?**

In [44]:
linreg_coef = pd.DataFrame(linreg.coef_, columns=['Coefficient']) # Ваш код тут
linreg_coef = linreg_coef.copy().applymap(abs)
linreg_coef = linreg_coef.sort_values(by='Coefficient',ascending=False) # Ваш код тут

linreg_coef


,Coefficient
7,227.551202
1,1.886874
8,0.995830
9,0.552495
4,0.352051
0,0.113731
3,0.108183
10,0.105264
5,0.002439
2,0.001485


## Lasso-регресія

**Навчіть Lasso-регресію з невеликим коефіцієнтом $\alpha = 0.01$ (слабка регуляризація). Нехай знову random_state=17.**

In [45]:
lasso1 = Lasso(alpha=0.01, random_state=17) # Ваш код тут
lasso1.fit(X_train, y_train) # Ваш код тут

Lasso(alpha=0.01, random_state=17)

**Подивіться на коефіцієнти моделі і відранжуйте ознаки за впливом на якість вина. Яка ознака є найменш важливою для пояснення цільового ознаки в моделі Lasso?**

In [46]:
lasso1_coef = pd.DataFrame(lasso1.coef_, columns=['Coefficient']) # Ваш код тут
lasso1_coef = lasso1_coef.copy().applymap(abs)
lasso1_coef = lasso1_coef.sort_values(by='Coefficient',ascending=False)
lasso1_coef

,Coefficient
1,0.967909
10,0.352083
0,0.059770
3,0.022231
5,0.005223
6,0.001108
2,0.000000
4,0.000000
7,0.000000
8,0.000000


**Тепер визначте краще значення $\alpha$ в процесі 5-кратної крос-валідації. Використовуйте LassoCV і random_state=17.**

In [47]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17) # Ваш код тут
lasso_cv.fit(X_train, y_train) # Ваш код тут

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [ ]:
lasso_cv.alpha_

np.float64(1e-06)

**Виведіть коефіцієнти "кращого" Lasso в порядку спадання впливу на якість вина.** 

**Запитання 3. (2 бали). Яка ознака "обнулилась першою" в налаштованій моделі LASSO?**

In [53]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_, columns=['alph']) # Ваш код тут
lasso_cv_coef = lasso_cv_coef.copy().applymap(abs)
lasso_cv_coef = lasso_cv_coef.sort_values(by='alph', ascending=False) # Ваш код тут
lasso_cv_coef

,alph
7,223.055887
1,1.888451
8,0.981016
9,0.546349
4,0.331071
10,0.110440
0,0.110345
3,0.106579
5,0.002466
2,0.002444


**Запитання 4. (1 бал). Які середньоквадратичні помилки налаштованої LASSO-регресії на навчальній і відкладеній вибірках?**

In [55]:
y_pred_train = lasso_cv.predict(X_train)
y_pred_test = lasso_cv.predict(X_holdout)
mse_train = mean_squared_error(y_pred=y_pred_train, y_true=y_train)
mse_train = mean_squared_error(y_pred=y_pred_test, y_true=y_holdout)
print(f"Mean squared error (train): {mse_train:.3f}") # Ваш код тут
print(f"Mean squared error (test): {mse_test:.3f}") # Ваш код тут

Mean squared error (train): 0.583
Mean squared error (test): 0.584


## Випадковий ліс

**Навчіть випадковий ліс з параметрами за замовчуванням, фіксуючи тільки random_state=17.**

In [56]:
forest = RandomForestRegressor(random_state=17) # Ваш код тут
forest.fit(X_train, y_train) # Ваш код тут

RandomForestRegressor(random_state=17)

**Запитання 5. (2 бали). Які середньоквадратичні помилки випадкового лісу на навчальній вибірці, на крос-валідації (cross_val_score з scoring='neg_mean_squared_error' і іншими параметрами за замовчуванням) та на відкладеній вибірці?**

### Test

In [57]:
y_pred = forest.predict(X_holdout)
mse_test = mean_squared_error(y_holdout, y_pred)
print(f'MSE на тестовій вибірці: {mse_test}')

MSE на тестовій вибірці: 0.37076367346938777


### Train

In [58]:
y_pred = forest.predict(X_train)
mse_train = mean_squared_error(y_train, y_pred)
print(f'MSE на тренувальній вибірці: {mse_train}')

MSE на тренувальній вибірці: 0.052711260210035006


### CV

In [59]:
mse_cv = cross_val_score(forest, X, y, cv=5, scoring='neg_mean_squared_error')
mse_cv = -mse_cv
print(f'Середнє MSE за результатами крос-валідації: {mse_cv.mean()}')

Середнє MSE за результатами крос-валідації: 0.5347123359738175


**Налаштуйте параметри min_samples_leaf і max_depth за допомогою GridSearchCV і знову перевірте якість моделі на крос-валідації і на відкладеній вибірці.**

In [62]:
forest_params = {'max_depth': list(range(10, 15)), 
                 'min_samples_leaf': list(range(1, 4)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(estimator=forest, param_grid=forest_params, cv=5, verbose=2, n_jobs=-1) # Ваш код тут
locally_best_forest.fit(X_train, y_train) # Ваш код тут

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=1; total time=   1.3s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=1; total time=   1.2s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=1; total time=   1.3s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=2; total time=   1.1s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=1; total time=   1.4s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=2; total time=   1.3s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=2; total time=   1.4s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=1; total time=   1.3s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=2; total time=   1.2s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=2; total time=   1.2s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=3; total time=   1.1s
[CV] END ...max_depth=10, max_features=6, min_s

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=17), n_jobs=-1,
             param_grid={'max_depth': [10, 11, 12, 13, 14],
                         'max_features': [6, 7, 8, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 3]},
             verbose=2)

In [63]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 14, 'max_features': 6, 'min_samples_leaf': 1},
 np.float64(0.473303437455091))

**Запитання 6. (1 бал). Які середньоквадратичні помилки налаштованого випадкового лісу на крос-валідації (cross_val_score з scoring='neg_mean_squared_error') та на відкладеній вибірці?**

In [65]:
y_pred_test = locally_best_forest.predict(X_holdout)
mse_test = mean_squared_error(y_holdout, y_pred_test)

mse_cv = cross_val_score(locally_best_forest, X, y, cv=5, scoring='neg_mean_squared_error')
mse_cv = -mse_cv

print(f'Середнє MSE за результатами крос-валідації: {mse_cv.mean()}')
print(f"Mean squared error (test): {mse_test: .3f}")

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=2; total time=   1.1s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=2; total time=   1.1s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=1; total time=   1.1s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=2; total time=   1.1s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=1; total time=   1.2s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=1; total time=   1.1s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=1; total time=   1.2s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=1; total time=   1.3s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=3; total time=   1.1s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=2; total time=   1.1s
[CV] END ...max_depth=10, max_features=6, min_samples_leaf=3; total time=   1.1s
[CV] END ...max_depth=10, max_features=6, min_s

**Оцініть важливість ознак за допомогою випадкового лісу.**

**Запитання 7. (1 бал). Яка ознака виявилась головною у налаштованій моделі випадкового лісу?**

In [70]:
best_forest = RandomForestRegressor(random_state=17, max_depth=14, max_features=6, min_samples_leaf=1)
best_forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=14, max_features=6, random_state=17)

In [75]:
rf_importance = pd.DataFrame(best_forest.feature_importances_, columns=['imp']) # Ваш код тут
rf_importance = rf_importance.sort_values(by='imp', ascending=False) # Ваш код тут
rf_importance

,imp
10,0.222185
1,0.121293
5,0.111569
7,0.087282
3,0.071716
6,0.071365
4,0.069642
8,0.067210
0,0.063376
2,0.059291


**Зробіть висновки про якість моделей і оцінки впливу ознак на якість вина за допомогою цих трьох моделей.**